In [18]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import os
import warnings

In [2]:
# ignore warnings for tweets incorrectly parsed due to them containing ;
warnings.filterwarnings('ignore')

df = None
folder = 'hydrated_tweets/'
for filename in os.listdir(folder):
    if filename == '.DS_Store':
            continue
    newname = folder+filename
    for name in os.listdir(newname):
        if name == '.DS_Store':
            continue
        f = os.path.join(newname, name)
        if df is None:
            df = pd.read_csv(f, sep=';', engine='python',quotechar='"', error_bad_lines=False,warn_bad_lines=False)
        else:
            tempdf = pd.read_csv(f, sep=';',engine='python',quotechar='"', error_bad_lines=False,warn_bad_lines=False)
            df = pd.concat([df,tempdf], ignore_index=True)

print(df.keys())
print(df.head())


Index(['id', 'text', 'location', 'place', 'country', 'retweeted',
       'retweet_count', 'favorited', 'favorite_count', 'language', 'date',
       'verified', 'verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,'],
      dtype='object')
                    id                                               text  \
0  1319111210864939011  RT @RepValDemings: Amy Coney Barrett ruled tha...   
1  1319111212194549761  RT @realDonaldTrump: Just don’t see any way Na...   
2  1319111212672667648  RT @ddale8: It turns out Trump has been making...   
3  1319111212743815169  RT @Laurie_Garrett: This would literally amoun...   
4  1319111212794249216  RT @JackPosobiec: I have never cursed on Twitt...   

           location place country retweeted retweet_count favorited  \
0            Kansas   NaN     NaN     False          3450     False   
1    Pittsburgh, PA   NaN     NaN     False         17984     False   
2     Ridgewood, NJ   NaN     NaN     False           462     False   
3     Virginia, USA   NaN     

In [ ]:
df.to_csv(f'alltweets.csv',sep=';',index=False,header=True,encoding='utf-8')

In [54]:
#alltweetsdf = pd.read_csv('alltweets.csv',sep=';')
alltweetsdf = df
alltweetsdf.head()

,id,text,location,place,country,retweeted,retweet_count,favorited,favorite_count,language,date,verified,"verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
0,1319111210864939011,RT @RepValDemings: Amy Coney Barrett ruled tha...,Kansas,NaN,NaN,False,3450,False,0,en,10-22-03,False,NaN
1,1319111212194549761,RT @realDonaldTrump: Just don’t see any way Na...,"Pittsburgh, PA",NaN,NaN,False,17984,False,0,en,10-22-03,False,NaN
2,1319111212672667648,RT @ddale8: It turns out Trump has been making...,"Ridgewood, NJ",NaN,NaN,False,462,False,0,en,10-22-03,False,NaN
3,1319111212743815169,RT @Laurie_Garrett: This would literally amoun...,"Virginia, USA",NaN,NaN,False,1580,False,0,en,10-22-03,False,NaN
4,1319111212794249216,RT @JackPosobiec: I have never cursed on Twitt...,Fort Worth Texas,NaN,NaN,False,1857,False,0,en,10-22-03,False,NaN


In [55]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))

(22305097, 13)
id                                             3
text                                       40927
location                                  100808
place                                   21995509
country                                 21992149
retweeted                                  41359
retweet_count                              41359
favorited                                  41359
favorite_count                             41359
language                                   41359
date                                       41363
verified                                  134637
verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,    22215236
dtype: int64


In [56]:
# remove incorrectly parsed rows and tweets which aren't english
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['verified'].isnull()==True)].index)
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['text'].isnull()==True)].index)
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['language']!='en')].index)

In [57]:
# dropping certain columns
print((alltweetsdf['retweeted']==True).sum(axis = 0))
print((alltweetsdf['favorited']==True).sum(axis = 0))
alltweetsdf = alltweetsdf.drop(columns=['retweeted','favorited','language','verified,,,,,,,,,,,,,,,,,,,,,,,,,,,,'])

0
0


In [58]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))
# bit more than half the tweets are retweets
print('Retweets:',alltweetsdf['text'].duplicated().sum())

(19303737, 9)
id                       0
text                     0
location             43067
place             19066901
country           19066921
retweet_count            0
favorite_count           0
date                     0
verified                 0
dtype: int64
Retweets: 12291948


In [59]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New York', 'New Mexico', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
stateCodes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
stateMapping = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NY': 'New York', 'NM': 'New Mexico', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT':  'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV':  'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming','D.C.':'District of Columbia'}

In [69]:
# some people have multiple states in their description, first state check is their given state
def findstate(x):
    location = x[0]
    place = x[1]
    for state in stateMapping:
        # seperate if statement for DC to check for multiple ways people write DC
        if state == 'D.C.':
            if f', {state}' in str(location) or stateMapping[state] in str(location) or f', {state}' in str(place) or stateMapping[state] in str(place) or ', DC' in str(location) or ', DC' in str(place):
                return stateMapping[state]
        elif f', {state}' in str(location) or stateMapping[state] in str(location) or f', {state}' in str(place) or stateMapping[state] in str(place):
            # checks that washington DC is not read as the state washington
            if state == 'WA' and 'D.C.' in str(location) or 'D.C.' in str(place) or ', DC' in str(location) or ', DC' in str(place):
                continue
            return stateMapping[state]
    return None
alltweetsdf["state"] = np.nan
alltweetsdf["state"] = alltweetsdf.apply(lambda row : findstate([row['location'],row['place']]), axis=1)

In [70]:
alltweetsdf.head(50)

,id,text,location,place,country,retweet_count,favorite_count,date,verified,state
0,1319111210864939011,RT @RepValDemings: Amy Coney Barrett ruled tha...,Kansas,NaN,NaN,3450,0,10-22-03,False,Kansas
1,1319111212194549761,RT @realDonaldTrump: Just don’t see any way Na...,"Pittsburgh, PA",NaN,NaN,17984,0,10-22-03,False,Pennsylvania
2,1319111212672667648,RT @ddale8: It turns out Trump has been making...,"Ridgewood, NJ",NaN,NaN,462,0,10-22-03,False,New Jersey
3,1319111212743815169,RT @Laurie_Garrett: This would literally amoun...,"Virginia, USA",NaN,NaN,1580,0,10-22-03,False,Virginia
4,1319111212794249216,RT @JackPosobiec: I have never cursed on Twitt...,Fort Worth Texas,NaN,NaN,1857,0,10-22-03,False,Texas
5,1319111212831887360,RT @Amy_Siskind: Note every bit of foreign int...,Colorado,NaN,NaN,694,0,10-22-03,False,Colorado
6,1319111212853022721,RT @donwinslow: Hunter Biden is not responsibl...,"Boston, MA",NaN,NaN,5215,0,10-22-03,False,Massachusetts
7,1319111213087924226,RT @willchamberlain: Maybe *you* never lived u...,One-Party-Rule California,NaN,NaN,602,0,10-22-03,False,California
8,1319111213222141952,RT @zachsilberberg: what a difficult week for ...,"Toronto, Ontario",NaN,NaN,586,0,10-22-03,False,None
9,1319111213238964226,RT @TCRG2012: @cynthia_ann @LisaWNC1 @Kelleyro...,Western Massachusetts,NaN,NaN,2,0,10-22-03,False,Massachusetts


In [71]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))
print('Retweets:',alltweetsdf['text'].duplicated().sum())

(19303737, 10)
id                       0
text                     0
location             43067
place             19066901
country           19066921
retweet_count            0
favorite_count           0
date                     0
verified                 0
state              9848184
dtype: int64
Retweets: 12291948


In [72]:
# delete nan rows from states
# delete location place and country columns
alltweetsdf = alltweetsdf.drop(alltweetsdf[(alltweetsdf['state'].isnull()==True)].index)
alltweetsdf = alltweetsdf.drop(columns=['location','place','country'])

In [73]:
print(alltweetsdf.shape)
print(alltweetsdf.isnull().sum(axis = 0))
print('Retweets:',alltweetsdf['text'].duplicated().sum())

(9455553, 7)
id                0
text              0
retweet_count     0
favorite_count    0
date              0
verified          0
state             0
dtype: int64
Retweets: 5919982


In [74]:
alltweetsdf['state'].value_counts()

California              1312674
Texas                    936219
Florida                  835746
New York                 666869
Pennsylvania             331117
Ohio                     297217
Illinois                 282975
Washington               269193
Arizona                  265532
Georgia                  264461
Michigan                 261664
North Carolina           246036
Virginia                 241849
New Jersey               208442
Massachusetts            208283
Colorado                 197602
Oregon                   182661
Tennessee                169081
District of Columbia     167601
Maryland                 151955
Minnesota                148411
Indiana                  135123
Wisconsin                123221
South Carolina           114771
Nevada                   111810
Missouri                 107252
Alabama                  103370
Kentucky                  95182
Louisiana                 92316
Oklahoma                  86047
Kansas                    84889
Connecti

In [75]:
word_count = Counter(" ".join(alltweetsdf['text']).split()).most_common(10)
word_frequency = pd.DataFrame(word_count, columns = ['Word', 'Frequency'])
print(word_frequency)

    Word  Frequency
0     RT    6685883
1    the    5317061
2     to    4174267
3      a    2847701
4    and    2709938
5     of    2498969
6     is    2466279
7     in    2435311
8  Trump    1986047
9    for    1835860


In [80]:
stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from", "rt", 'this','that','is','are','you','i','it','we','be','have']

def clean_tweet(tweet):
    temp = tweet.lower()
    temp = re.sub(r'http\S+', '', temp) # removes links
    temp = re.sub('[().,:!?]', ' ', temp)  # removes punctutation
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords] # removes stopwords
    temp = " ".join(word for word in temp)
    return temp

In [81]:
cleantweets = alltweetsdf
cleantweets["filtered_text"] = np.nan
cleantweets["filtered_text"] = cleantweets.apply(lambda row : clean_tweet(row['text']), axis=1)

In [82]:
cleantweets.to_csv(f'cleantweets.csv',sep=';',index=False,header=True,encoding='utf-8')
cleantweets.head()

,id,text,retweet_count,favorite_count,date,verified,state,filtered_text
0,1319111210864939011,RT @RepValDemings: Amy Coney Barrett ruled tha...,3450,0,10-22-03,False,Kansas,@repvaldemings amy coney barrett ruled supervi...
1,1319111212194549761,RT @realDonaldTrump: Just don’t see any way Na...,17984,0,10-22-03,False,Pennsylvania,@realdonaldtrump just don’t see any way nancy ...
2,1319111212672667648,RT @ddale8: It turns out Trump has been making...,462,0,10-22-03,False,New Jersey,@ddale8 turns out trump has been making false ...
3,1319111212743815169,RT @Laurie_Garrett: This would literally amoun...,1580,0,10-22-03,False,Virginia,@laurie_garrett would literally amount partisa...
4,1319111212794249216,RT @JackPosobiec: I have never cursed on Twitt...,1857,0,10-22-03,False,Texas,@jackposobiec never cursed twitter before but ...


In [83]:
word_count = Counter(" ".join(cleantweets['filtered_text']).split()).most_common(10)
word_frequency = pd.DataFrame(word_count, columns = ['Word', 'Frequency'])
print(word_frequency)

               Word  Frequency
0             trump    2584138
1  @realdonaldtrump    1524905
2             biden    1506394
3              vote     955524
4         @joebiden     954235
5                he     953744
6               not     931665
7              with     791699
8              will     787633
9                if     782228
